**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [47]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [48]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [49]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [50]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [51]:
df_supply

,0
0,651
1,256
2,786
3,728
4,947
...,...
245,684
246,792
247,718
248,930


In [52]:
df_demand

,0
0,812
1,334
2,786
3,557
4,295
...,...
95,284
96,383
97,116
98,917


In [53]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,95.340041,77.787079,65.791289,95.979939,80.828502,74.110566,129.038746,54.337982,64.740600,32.606007,...,36.674434,56.133304,86.977116,40.865226,47.749091,61.997349,43.008016,46.420652,63.537457,51.183003
1,96.075280,77.985277,66.582292,95.264938,81.536972,73.930342,129.062084,53.597530,65.569346,31.795567,...,37.251645,56.134120,87.690178,40.683569,47.278052,62.503675,42.983572,47.673403,63.575423,51.015241
2,89.216991,76.036570,59.560866,100.985287,75.042950,75.321673,130.630763,60.089269,58.222158,39.802675,...,33.832238,58.806145,81.093363,43.365011,51.844836,58.154851,44.206215,36.646905,65.728794,52.948793
3,88.909846,77.250197,59.375853,103.694568,74.925359,77.312010,129.884835,63.004863,57.931464,42.955093,...,35.407794,58.701899,80.909460,45.892951,54.553804,58.857920,46.467697,34.854543,65.407175,55.213748
4,90.332438,78.305996,60.774417,103.968059,76.317127,78.041941,128.658808,63.083934,59.345614,42.697049,...,36.293153,57.318957,82.312895,46.296642,54.824385,60.064721,47.039389,36.330642,64.068564,55.788174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,92.105163,76.849154,62.425281,98.749738,77.745124,74.759558,129.554384,57.397095,61.219545,36.296852,...,34.705912,57.048045,83.855254,41.944185,49.825505,59.868845,43.394655,41.161723,64.228588,51.992475
246,90.080982,76.697772,60.419487,101.163977,75.893422,75.788317,129.908485,60.157636,59.088756,39.660966,...,34.453897,58.002503,81.948644,43.657709,52.041154,58.912943,44.604334,37.508925,64.951263,53.336035
247,91.160315,76.842298,61.475138,100.028535,76.882344,75.342392,129.607647,58.831148,60.205381,38.010848,...,34.569381,57.373880,82.967283,42.831614,50.980412,59.444318,44.030595,39.360296,64.443252,52.712152
248,92.929593,78.578603,63.246548,101.135774,78.665239,76.880721,127.852461,59.799286,61.965001,38.675363,...,36.319561,55.578686,84.746268,44.202759,52.179882,61.239851,45.534937,40.725504,62.659371,54.179541


In [54]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [55]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

142719 57492


In [56]:
m = gp.Model("probrema")

In [57]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [58]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [59]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [60]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [61]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [62]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [63]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1500 rows, 112500 columns and 400000 nonzeros
Model fingerprint: 0x19556f5c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+04]
Presolve removed 550 rows and 0 columns
Presolve time: 0.16s
Presolved: 950 rows, 112500 columns, 312500 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 2.003e+05
 Factor NZ  : 2.804e+05 (roughly 50 MB of memory)
 Factor Ops : 1.013e+08 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.8

In [64]:
m.write("out.sol")